In [42]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

#tb.loc[tb['Instrument Classification (ISO 10962 CFI code)'].isin(['JFTXFP'])]
#tb.plot.hist(tb.groupby(['Trading platform']).Price.agg(['mean', max]))
#tb.describe()
#print(tb.shape)

df = pd.read_csv('C:/Users/inald/Documents/NWMprod/wetransfer-0f4d76/rts27.csv')

#Columns deemed unneccessary for this task
df = df.drop(['file_name','designated_market_makers','legal_entity_name', 'systematic_internaliser_flag'], axis = 1)

df2 = df
#print(df.shape)
#df.head(20)

C:\Users\inald\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (6,17,18,19,21,22,24,32,36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
#Columns deemed unneccessary for this task
#df = df.drop(['file_name','designated_market_makers','legal_entity_name', 'systematic_internaliser_flag'], axis = 1)
df = df2
#Filters the code to FX forwards only
#df = df.loc[(df['source_instrument_desc']).astype(str).str.contains("Forward")]
df = df.loc[(df['instrument_classification']).astype(str).str.contains("JF") | (df['source_instrument_desc']).astype(str).str.contains("Forward")]

#Drops the columns of which half of the fields have no data.
#df = df.dropna(axis=1, thresh=2000000)

df

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Select target
y = df['date_created']
# To keep things simple, we'll use only numerical predictors
predictors = df.drop(['date_created'], axis=1)
X = df

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [5]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer(missing_values='np.nan', strategy = 'most_frequent')
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print(imputed_X_train.columns.isnull().sum())

MemoryError: Unable to allocate 2.88 GiB for an array with shape (37, 10451791) and data type object

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
for col in object_cols:
    label_encoder.fit(pd.concat([X_train[col], X_valid[col]], axis=0, sort=False))
    label_X_train[col] = label_encoder.transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])